# 유튜브 조회수 기반 데이터 수집

이 노트북은 기존에 수집된 유튜브 데이터에서 제공된 `video_id`를 바탕으로, 각 영상의 조회수를 크롤링하여 수집하는 과정에 중점을 둡니다. 수집된 조회수 데이터는 이후 조회수 기반의 연관어 추출을 위해 사용됩니다.

## 데이터 수집 방식

1. **기존 데이터 활용**: 이미 확보된 유튜브 데이터에서 각 영상의 `video_id`를 추출합니다.
2. **조회수 수집**: 추출된 `video_id`를 통해 유튜브 영상의 조회수를 크롤링하여 수집합니다.
3. **크롤링 방법**: YouTube 페이지를 자동으로 탐색하고, 각 영상의 조회수를 가져오는 방식으로 구현됩니다.

In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# 크롬 드라이버 설정 (크롬 드라이버 경로 수정 필요)
chrome_driver_path = 'path/to/chromedriver'  # 크롬 드라이버 경로를 설정하세요.
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service)

In [ ]:
def get_video_data(video_url):
    """
    주어진 유튜브 비디오 URL에서 비디오 정보를 수집하는 함수입니다.

    Args:
        video_url (str): 비디오의 URL

    Returns:
        dict: 비디오 정보가 포함된 딕셔너리
    """
    # 유튜브 비디오 페이지 열기
    driver.get(video_url)
    time.sleep(2)  # 페이지 로드 대기

    # 페이지의 HTML 파싱
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # 비디오 정보 수집을 위한 딕셔너리 초기화
    video_data = {}

    # 비디오 ID 추출
    video_data['video_id'] = video_url.split('v=')[-1]

    # 조회수 추출
    views_element = soup.find('span', class_='view-count style-scope yt-view-count-renderer')
    video_data['video_views'] = views_element.text.strip() if views_element else None  # 조회수가 없을 경우 None

    return video_data


In [ ]:
# 유튜브 데이터 불러오기
df = pd.read_csv("../data/video_sample_20241013.csv", encoding="utf-8-sig", index_col=0)
# 비디오 URL 리스트
df['video_urls'] = df.video_id.apply(lambda x: f'https://www.youtube.com/watch?v={x}')

In [ ]:
# 비디오 데이터 수집을 위한 리스트 초기화
video_data_list = []

# 각 비디오 URL에 대해 데이터 수집
for url in df['video_urls'] :
    video_data = get_video_data(url)  # 비디오 데이터 수집
    video_data_list.append(video_data)  # 수집한 데이터 리스트에 추가


In [ ]:
# 수집된 데이터를 DataFrame으로 변환
df = pd.DataFrame(video_data_list)

# DataFrame을 CSV 파일로 저장
df.to_csv('../data/video_sample.csv', index=False, encoding='utf-8-sig')


In [ ]:
# 드라이버 종료
driver.quit()